Chapter 9 of IBM's data science certification: week 1

In [42]:
import pandas as pd
import numpy as np
import requests 
!pip install bs4
from bs4 import BeautifulSoup
import urllib.request
#!pip install lxml
!pip install geocoder
import geocoder

In [64]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#postal code csv
PostalCodes = pd.read_csv('Geospatial_Coordinates.csv')
PostalCodes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [44]:
response = requests.get(url= URL)
print(response.status_code)
soup = BeautifulSoup(response.content, 'html.parser')
right_table = soup.find_all("table", class_="wikitable sortable")
title = soup.find(id="firstHeading")
print(title.text)

200
List of postal codes of Canada: M


In [45]:
A=[]
B=[]
C=[]

for row in right_table[0].findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

df=pd.DataFrame(A, columns = ['Postal Code'])
df['Borough'] = B
df['neighbourhood'] = C
df['Postal Code'] = df['Postal Code'].str.replace('\n', '')
df['Borough'] = df['Borough'].str.replace('\n', '')
df['neighbourhood'] = df['neighbourhood'].str.replace('\n', '')
df = df.loc[df['Borough'] != 'Not assigned']


In [46]:
df = df.groupby('Postal Code').agg( ', '.join).reset_index()
df['neighbourhood']=df['Borough'].where(df['neighbourhood'].eq('Not Assigned'),df['neighbourhood'])
df.to_csv('df.csv')

print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0])
      )

The dataframe has 10 boroughs and 103 neighbourhoods.


# Add on lat and long per postal code

In [47]:
# #This didn't work. Called >1000 times, didn't get any output
# df['Latitude'] = ''
# df['Longitude'] = ''
# for row in df.index:
#     PostalCode = df.loc[row, 'Postal Code']
#     print(PostalCode)
#     lat_lng_coords = None
#     count = 0
#     while (lat_lng_coords is None):
#         print(count)
#         count = count + 1
#         g = geocoder.google('{}, Toronto, Ontario'.format(PostalCode))
#         lat_lng_coords = g.latlng
#     df.loc[row, 'Latitude'] = lat_lng_coords[0]
#     df.loc[row, 'Longitude'] = lat_lng_coords[1]
#     print(df[row])

df = df.merge(PostalCodes,
              how = 'left',
              left_on  = 'Postal Code',
              right_on = 'Postal Code')
neighbourhoods = df
neighbourhoods

,Postal Code,Borough,neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# Start analyzing the neighbourhood with 4 square

In [48]:

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.



In [49]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# show buroughs on a folium map

In [50]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version


In [51]:
CLIENT_ID = 'DICSJ5BRJONYYQV0OGU314E312WO2C0H30GCFJ5GJAD2HESW' # your Foursquare ID
CLIENT_SECRET = 'HDKKFGO1OQL5TW0WCCGM5Z2KP4LVVLOYKROIWB55FGVXSKFK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DICSJ5BRJONYYQV0OGU314E312WO2C0H30GCFJ5GJAD2HESW
CLIENT_SECRET:HDKKFGO1OQL5TW0WCCGM5Z2KP4LVVLOYKROIWB55FGVXSKFK


## Get idea of surrounding venues by hood

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                  'neighbourhood Latitude', 
                  'neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
toronto_venues = getNearbyVenues(names=neighbourhoods['neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [54]:
print(toronto_venues.shape)
toronto_venues.head()


#toronto_venues.groupby(['neighbourhood','Venue Category'])['Venue Category'].count()

(2121, 7)


,neighbourhood,neighbourhood Latitude,neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [81]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['neighbourhood'] = toronto_venues['neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('neighbourhood').sum().reset_index()
toronto_grouped

,neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
92,"Willowdale, Willowdale West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94,Woodbine Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
num_top_venues = 5

for hood in toronto_grouped['neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   1.0
1                     Lounge   1.0
2               Skating Rink   1.0
3  Latin American Restaurant   1.0
4          Accessories Store   0.0


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   2.0
1             Pub   1.0
2  Sandwich Place   1.0
3    Dance Studio   1.0
4     Coffee Shop   1.0


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank   2.0
1                Coffee Shop   2.0
2               Intersection   1.0
3  Middle Eastern Restaurant   1.0
4              Shopping Mall   1.0


----Bayview Village----
                 venue  freq
0  Japanese Restaurant   1.0
1                 Bank   1.0
2   Chinese Restaurant   1.0
3                 Café   1.0
4    Accessories Store   0.0


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop   2.0
1           Sandw

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['neighbourhood'] = toronto_grouped['neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Escape Room,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dim Sum Restaurant,Dumpling Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pub,Dance Studio,Gym,Pharmacy,Dumpling Restaurant,Drugstore,Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Shopping Mall,Sandwich Place,Diner,Deli / Bodega,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Thai Restaurant,Juice Bar,Fast Food Restaurant,Butcher,Café,Restaurant


### Lets start clustering these together

In [85]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 3, 0, 3, 0, 0], dtype=int32)

In [86]:
# add clustering labels
toronto_grouped_clustering

,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
92,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('neighbourhood'), on='neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Bank,Breakfast Spot,Electronics Store,Medical Center,Restaurant,Rental Car Location,Dog Run,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Bank,Donut Shop,Doner Restaurant


## Lets take a look at these clusters

In [88]:
#cluster 0 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",0.0,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",0.0,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,Scarborough,"Guildwood, Morningside, West Hill",0.0,Intersection,Bank,Breakfast Spot,Electronics Store,Medical Center,Restaurant,Rental Car Location,Dog Run,Diner,Discount Store
3,Scarborough,Woburn,0.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,Cedarbrae,0.0,Hakka Restaurant,Bakery,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Bank,Donut Shop,Doner Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,York,Weston,0.0,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
99,Etobicoke,Westmount,0.0,Pizza Place,Coffee Shop,Sandwich Place,Chinese Restaurant,Discount Store,Middle Eastern Restaurant,Intersection,Donut Shop,Doner Restaurant,Dessert Shop
100,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",0.0,Pizza Place,Sandwich Place,Park,Bus Line,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
101,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",0.0,Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pharmacy,Coffee Shop,Sandwich Place,Pizza Place,Aquarium


In [89]:
#cluster 1 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Downtown Toronto,Church and Wellesley,1.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Mediterranean Restaurant,Café,Pub,Yoga Studio,Hotel
57,Downtown Toronto,Central Bay Street,1.0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Thai Restaurant,Seafood Restaurant


In [90]:
#cluster 2 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Downtown Toronto,"Richmond, Adelaide, King",2.0,Coffee Shop,Café,Restaurant,Clothing Store,Bakery,Hotel,Thai Restaurant,Gym,Deli / Bodega,Pizza Place
59,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",2.0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Italian Restaurant,Brewery,Scenic Lookout,Fried Chicken Joint,Park
60,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",2.0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Salad Place,American Restaurant,Seafood Restaurant,Italian Restaurant,Bakery
61,Downtown Toronto,"Commerce Court, Victoria Hotel",2.0,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Cocktail Bar,Deli / Bodega,Japanese Restaurant
69,Downtown Toronto,Stn A PO Boxes,2.0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Restaurant,Café,Beer Bar,Japanese Restaurant,Bakery,Breakfast Spot
70,Downtown Toronto,"First Canadian Place, Underground city",2.0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Gym,Asian Restaurant,Salad Place,Deli / Bodega,Steakhouse


In [91]:
#cluster 3 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,East York,Leaside,3.0,Coffee Shop,Sporting Goods Shop,Bank,Shopping Mall,Burger Joint,Furniture / Home Store,Sports Bar,Sushi Restaurant,Supermarket,Fish & Chips Shop
41,East Toronto,"The Danforth West, Riverdale",3.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Liquor Store,Indian Restaurant,Japanese Restaurant,Juice Bar,Spa
43,East Toronto,Studio District,3.0,Coffee Shop,Brewery,Gastropub,Bakery,Café,American Restaurant,Yoga Studio,Convenience Store,Seafood Restaurant,Cheese Shop
47,Central Toronto,Davisville,3.0,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Sushi Restaurant,Thai Restaurant,Café,Italian Restaurant,Gym,Park
51,Downtown Toronto,"St. James Town, Cabbagetown",3.0,Coffee Shop,Café,Restaurant,Italian Restaurant,Bakery,Pub,Market,Chinese Restaurant,Pizza Place,Pet Store
53,Downtown Toronto,"Regent Park, Harbourfront",3.0,Coffee Shop,Bakery,Park,Theater,Café,Breakfast Spot,Pub,Beer Store,Bank,French Restaurant
55,Downtown Toronto,St. James Town,3.0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Hotel,Moroccan Restaurant,Creperie,Park,Clothing Store
56,Downtown Toronto,Berczy Park,3.0,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Cheese Shop,Pharmacy,Restaurant,Farmers Market,Beer Bar,Shopping Mall
66,Downtown Toronto,"University of Toronto, Harbord",3.0,Café,Bar,Japanese Restaurant,Bookstore,Bakery,Yoga Studio,Restaurant,Sushi Restaurant,Beer Bar,Beer Store
67,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",3.0,Café,Coffee Shop,Bar,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Park,Caribbean Restaurant,Burger Joint,Gaming Cafe


In [92]:
#cluster 4 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,North York,"Fairview, Henry Farm, Oriole",4.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Shoe Store,Japanese Restaurant,Chinese Restaurant,Bakery,Juice Bar,Toy / Game Store
54,Downtown Toronto,"Garden District, Ryerson",4.0,Coffee Shop,Clothing Store,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Hotel,Café,Theater
